<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Реализуем триграмную модель

In [0]:
from google.colab import files
uploaded = files.upload()

Saving anna_karenina.txt to anna_karenina.txt
Saving besy_dostoevsky.txt to besy_dostoevsky.txt


In [0]:
dostoevsky = open('besy_dostoevsky.txt').read()
tolstoy = open('anna_karenina.txt').read()

In [210]:
print("Длина Бесов Достоевского -", len(dostoevsky))
print("Длина Анны Карениной Толстого - ", len(tolstoy))

Длина Бесов Достоевского - 1293557
Длина Анны Карениной Толстого -  1710408


In [0]:
from string import punctuation
import numpy as np

def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

In [213]:
print("Длина Бесов Достоевского в токенах -", len(norm_dostoevsky))
print("Длина Анны Карениной Толстого в токенах - ", len(norm_tolstoy))

Длина Бесов Достоевского в токенах - 208453
Длина Анны Карениной Толстого в токенах -  281201


In [0]:
from collections import Counter
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [215]:
import nltk
nltk.download('punkt')
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_tolstoy = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))

unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

cut = sentences_tolstoy[:1000]
for sentence in cut:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence))

In [0]:
from collections import Counter
unigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    trigrams_dostoevsky.update(ngrammer(sentence, n=3))
    
unigrams_tolstoy = Counter()
trigrams_tolstoy = Counter()

for sentence in cut:
    unigrams_tolstoy.update(sentence)
    trigrams_tolstoy.update(ngrammer(sentence, n=3))

In [218]:
print(len(unigrams_dostoevsky))

32474


In [219]:
print(trigrams_tolstoy.most_common(10))

[('<start> <start> –', 393), ('<start> <start> он', 41), ('<start> <start> и', 36), ('– сказал он', 32), ('<start> <start> степан', 30), ('<start> степан аркадьич', 30), ('<start> <start> но', 28), ('<start> – ну', 28), ('<start> <start> она', 27), ('<start> <start> левин', 26)]


In [0]:
matrix_tolstoy = np.zeros((len(unigrams_tolstoy), 
                   len(unigrams_tolstoy)))

id2word_unigrams = list(unigrams_tolstoy)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}
id2word_bigrams = list(bigrams_tolstoy)
word2id_bigrams = {coll:d for d, coll in enumerate(id2word_bigrams)}

for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_unigrams[word1]][word2id_unigrams[word2]] =  (bigrams_tolstoy[ngram]/
                                                                     unigrams_tolstoy[word1])

In [0]:
matrix_tolstoy = np.zeros((len(bigrams_tolstoy), 
                   len(unigrams_tolstoy)))

for ngram in trigrams_tolstoy:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_tolstoy[word2id_bigrams[bigram]][word2id_unigrams[word3]] =  (trigrams_tolstoy[ngram]/
                                                                     bigrams_tolstoy[bigram])

In [0]:
def generate(matrix, id2word, word2id, n=100, start=['<start>','<start>']):
    text = []
    current_bigram = start
    current_idx = word2id[' '.join(current_bigram)]
    
    for i in range(n):  
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_bigram = start
            current_idx = word2id[' '.join(current_bigram)]
        else:
          current_bigram = [current_bigram[1], id2word[chosen]]
          current_idx = word2id[' '.join(current_bigram)]


    
    return ' '.join(text)

In [223]:
print(generate(matrix_tolstoy, id2word_unigrams, word2id_bigrams).replace('<end>', '\n'))

– как не понимаю что вы делаете – сказал он 
 – да нехорошо 
 но ведь здесь этого нет жизни 
 ах сколько мыслей 
 хотя он имел большое уважение к своему сердцу 
 – это урок смирения 
 – мы русские всегда так 
 но чем же кончил он с благодарностью взглянул на облонского 
 степан аркадьич весело улыбался 
 iv дарья александровна приказали доложить что они связывали научные вопросы с задушевными несколько раз почти подходили к этим вопросам но каждый раз как только они подходили близко к самому главному опять отходят и он счастлив 
 – виновато


2. Gensim.models.Phrases

In [208]:
from nltk.tokenize import sent_tokenize
 
text="""В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**. Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу. Она считалась богатой невестою, и многие прочили ее за себя или за сыновей. Марья Гавриловна была воспитана на французских романах, и следственно была влюблена. Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне."""
sentences = sent_tokenize(text)


for sents in sentences:
    print(sents)

В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**.
Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу.
Она считалась богатой невестою, и многие прочили ее за себя или за сыновей.
Марья Гавриловна была воспитана на французских романах, и следственно была влюблена.
Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне.


In [200]:
phrases = gensim.models.Phrases(texts, min_count=1, scoring = 'npmi', threshold=0)
p_2 = gensim.models.Phrases(phrases[voc], scoring='npmi', threshold=-1)
p_3 = gensim.models.Phrases(p_[phrases[voc]], scoring='npmi', threshold=-1)
p_4 = gensim.models.Phrases(p3[p2[phrases[voc]]], scoring='npmi', threshold=-1)
print(list(p_4(p_3(p_2[voc]))))

NameError: ignored